In [1]:

import sys
sys.path.append('/home/techt/Desktop/a4s')


### Step 1: Initial Data Loading

In [2]:
from utils.preprocess import *
# 1. Identify the paths to the soft file and the matrix file
cohort_dir = '/media/techt/DATA/GEO/Physical_Exercise_Response/GSE106865'
soft_file, matrix_file = geo_get_relevant_filepaths(cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"A longitudinal gene expression microarray analysis of skeletal muscle resistance training"
!Series_summary	"Skeletal muscle adapts to resistance exercise (RE) performance acutely and chronically. An important regulatory step of muscle adaptation to RE is gene expression. Microarray analysis can be used as an exploratory method to investigate how genes and gene clusters are modulated acutely and chronically by RE. The purpose of the present study was to investigate the effect of training status in the basal (rested) and pre- to 24h post-RE on the global transcriptome in vastus lateralis muscle biopsies of young men. Muscle biopsies of nine young men who undertook RE training for 10-wks were collected pre and 24h post-RE at the first (W1) and last (W10) weeks of training and analysed using microarray. An unaccustomed RE bout (at W1) up-regulated muscle gene transcripts related to stress (e.g., heat shock proteins), damage and inflammation, structura

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
is_gene_available = False
trait_row = age_row = gender_row = None  # set to different values when applicable
convert_trait = convert_age = convert_gender = None  # define the functions when applicable

# Determine if gene expression data is available
is_gene_available = True

# Variable Availability and Data Type Conversion
# Setting variables based on sample characteristics dictionary

trait_row = 2 if len(set(['untrained', 'trained'])) > 1 else None
age_row = None  # No age data found
gender_row = 1 if len(set(['male'])) > 1 else None

# Define conversion functions
def convert_trait(value):
    if ':' in value:
        trait_value = value.split(':')[-1].strip().lower()
        if trait_value == 'untrained':
            return 0
        elif trait_value == 'trained':
            return 1
    return None

def convert_age(value):
    return None  # No age data to convert

def convert_gender(value):
    if ':' in value:
        gender_value = value.split(':')[-1].strip().lower()
        if gender_value == 'male':
            return 1
        elif gender_value == 'female':
            return 0
    return None

# Save cohort information
save_cohort_info('GSE106865', './preprocessed/Physical_Exercise_Response/cohort_info.json', is_gene_available, trait_row is not None)

# If clinical data is available (i.e., trait_row is not None)
if trait_row is not None:
    selected_clinical_data = geo_select_clinical_features(clinical_data, 'Physical_Exercise_Response', trait_row, convert_trait, age_row, convert_age, gender_row, convert_gender)
    csv_path = './preprocessed/Physical_Exercise_Response/trait_data/GSE106865.csv'
    selected_clinical_data.to_csv(csv_path)
    print(preview_df(selected_clinical_data))


{'GSM2856160': [0], 'GSM2856161': [0], 'GSM2856162': [1], 'GSM2856163': [1], 'GSM2856164': [0], 'GSM2856165': [0], 'GSM2856166': [1], 'GSM2856167': [1], 'GSM2856168': [0], 'GSM2856169': [0], 'GSM2856170': [1], 'GSM2856171': [1], 'GSM2856172': [0], 'GSM2856173': [0], 'GSM2856174': [1], 'GSM2856175': [1], 'GSM2856176': [0], 'GSM2856177': [0], 'GSM2856178': [1], 'GSM2856179': [1], 'GSM2856180': [0], 'GSM2856181': [0], 'GSM2856182': [1], 'GSM2856183': [1], 'GSM2856184': [0], 'GSM2856185': [0], 'GSM2856186': [1], 'GSM2856187': [1], 'GSM2856188': [0], 'GSM2856189': [0], 'GSM2856190': [1], 'GSM2856191': [1], 'GSM2856192': [0], 'GSM2856193': [0], 'GSM2856194': [1], 'GSM2856195': [1], 'GSM2856196': [0], 'GSM2856197': [0], 'GSM2856198': [1], 'GSM2856199': [1]}


### Step 3: Gene Data Extraction

In [4]:
# 1. Use the get_genetic_data function from the library to get the gene_data from the matrix_file previously defined.
gene_data = get_genetic_data(matrix_file)

# 2. Print the first 20 row ids for the following step.
print(gene_data.index[:20])


Index(['ILMN_1343291', 'ILMN_1343295', 'ILMN_1651199', 'ILMN_1651209',
       'ILMN_1651210', 'ILMN_1651221', 'ILMN_1651228', 'ILMN_1651229',
       'ILMN_1651230', 'ILMN_1651232', 'ILMN_1651235', 'ILMN_1651236',
       'ILMN_1651237', 'ILMN_1651238', 'ILMN_1651249', 'ILMN_1651253',
       'ILMN_1651254', 'ILMN_1651259', 'ILMN_1651260', 'ILMN_1651262'],
      dtype='object', name='ID')


### Step 4: Gene Identifier Review

In [5]:
requires_gene_mapping = True


### Step 5: Gene Annotation (Conditional)

In [6]:
# 1. Use the 'get_gene_annotation' function from the library to get gene annotation data from the soft file.
gene_annotation = get_gene_annotation(soft_file)

# 2. Use the 'preview_df' function from the library to preview the data and print out the results.
print("Gene annotation preview:")
print(preview_df(gene_annotation))


Gene annotation preview:
{'ID': ['ILMN_1343048', 'ILMN_1343049', 'ILMN_1343050', 'ILMN_1343052', 'ILMN_1343059'], 'Species': [nan, nan, nan, nan, nan], 'Source': [nan, nan, nan, nan, nan], 'Search_Key': [nan, nan, nan, nan, nan], 'Transcript': [nan, nan, nan, nan, nan], 'ILMN_Gene': [nan, nan, nan, nan, nan], 'Source_Reference_ID': [nan, nan, nan, nan, nan], 'RefSeq_ID': [nan, nan, nan, nan, nan], 'Unigene_ID': [nan, nan, nan, nan, nan], 'Entrez_Gene_ID': [nan, nan, nan, nan, nan], 'GI': [nan, nan, nan, nan, nan], 'Accession': [nan, nan, nan, nan, nan], 'Symbol': ['phage_lambda_genome', 'phage_lambda_genome', 'phage_lambda_genome:low', 'phage_lambda_genome:low', 'thrB'], 'Protein_Product': [nan, nan, nan, nan, 'thrB'], 'Probe_Id': [nan, nan, nan, nan, nan], 'Array_Address_Id': [5090180.0, 6510136.0, 7560739.0, 1450438.0, 1240647.0], 'Probe_Type': [nan, nan, nan, nan, nan], 'Probe_Start': [nan, nan, nan, nan, nan], 'SEQUENCE': ['GAATAAAGAACAATCTGCTGATGATCCCTCCGTGGATCTGATTCGTGTAA', 'CCAT

### Step 6: Gene Identifier Mapping

In [7]:
# 1. Define the keys for identifiers and gene symbols
identifier_key = 'ID'
gene_symbol_key = 'Symbol'

# 2. Get the dataframe storing the mapping between probe IDs and genes
mapping_df = get_gene_mapping(gene_annotation, identifier_key, gene_symbol_key)

# 3. Apply the mapping to get the resulting gene expression dataframe
gene_data = apply_gene_mapping(gene_data, mapping_df)


### Step 7: Data Normalization and Merging

In [8]:
# 1. Normalize the obtained gene data with the 'normalize_gene_symbols_in_index' function from the library.
normalized_gene_data = normalize_gene_symbols_in_index(gene_data)
gene_csv_path = './preprocessed/Physical_Exercise_Response/gene_data/GSE106865.csv'
normalized_gene_data.to_csv(gene_csv_path)

# 2. Merge the clinical and genetic data with the 'geo_merge_clinical_genetic_data' function from the library.
merged_data = geo_merge_clinical_genetic_data(selected_clinical_data, normalized_gene_data)

# 3. Determine whether the trait and some demographic attributes in the data is severely biased, and remove biased attributes.
trait_biased, unbiased_merged_data = judge_and_remove_biased_features(merged_data, 'Physical_Exercise_Response')

# 4. Save the cohort information.
save_cohort_info('GSE106865', './preprocessed/Physical_Exercise_Response/cohort_info.json', True, True, trait_biased, merged_data)

if not trait_biased:
    # 5. If the trait is not severely biased, save the merged data to a csv file.
    csv_path = './preprocessed/Physical_Exercise_Response/GSE106865.csv'
    unbiased_merged_data.to_csv(csv_path)


For the feature 'Physical_Exercise_Response', the least common label is '0.0' with 20 occurrences. This represents 50.00% of the dataset.
The distribution of the feature 'Physical_Exercise_Response' in this dataset is fine.

